## Import Libraries

In [22]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

## Load Data

In [11]:
DATASET_PATH = "../data/raw/"

In [12]:
# Liste pour stocker les chemins des fichiers et leurs labels
file_paths = []
labels = []

# Map des émotions pour RAVDESS
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# Parcours des fichiers dans le dossier
for root, dirs, files in os.walk(DATASET_PATH):
    for file in files:
        if file.endswith(".wav"):
            # Ajouter le chemin complet du fichier
            file_paths.append(os.path.join(root, file))
            
            # Extraire l'émotion à partir du nom du fichier
            emotion_code = file.split("-")[2]  # Le 3ème segment du nom correspond à l'émotion
            labels.append(emotion_map[emotion_code])

In [13]:
print(len(file_paths))

1440


In [14]:
print(len(labels))

1440


## Simple Encoding

In [34]:
def extract_features_simple(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # Charger le fichier audio avec son taux d'échantillonnage d'origine
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)  # Calculer 40 coefficients MFCC
    mfccs_scaled = np.mean(mfccs.T, axis=0)  # Moyenne sur le temps pour réduire la dimensionnalité
    return mfccs_scaled

In [35]:
features = []
for file_path in file_paths:
    features.append(extract_features_simple(file_path))

In [36]:
print("Taille du vecteur représentant un fichier :", len(features[1]))
print("Nombre de fichiers :", len(features))

Taille du vecteur représentant un fichier : 40
Nombre de fichiers : 1440


In [37]:
# Convertir les données en DataFrame
data = pd.DataFrame(features)
data['label'] = labels

data.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,-726.217224,68.541420,3.293398,12.205300,5.510278,13.667408,-2.983829,3.098029,-3.310813,-1.564384,...,-1.399109,-2.926855,0.013957,-0.490734,-0.570905,0.040399,-1.207217,-1.594982,-1.436487,neutral
1,-719.128296,70.201569,1.168397,13.122543,7.836950,14.411290,-4.111360,4.468973,-3.539367,-3.658607,...,-2.521470,-2.987673,0.409735,-0.484184,-1.398391,0.255203,-0.984978,-2.093061,-1.040791,neutral
2,-714.995728,69.689346,3.924564,11.924190,6.421723,11.011614,-2.878103,4.509558,-4.476109,-2.671550,...,-0.909152,-3.045955,-0.373294,-0.849145,-0.922105,-0.170320,-1.144423,-1.725613,-1.450561,neutral
3,-710.975281,67.564880,5.782241,13.230727,6.190846,12.628252,-1.675170,5.657494,-4.950634,-3.477545,...,-1.329651,-2.513405,-0.190276,-0.645949,-0.553919,0.459299,-1.580085,-1.647682,-1.509511,neutral
4,-759.921753,75.783524,6.023605,14.557394,6.454188,14.631508,-3.004551,4.620970,-5.200016,-0.707430,...,-2.188582,-2.835501,0.463746,-1.019167,-1.411441,0.350433,-1.519892,-1.250112,-0.613852,calm


In [38]:
# Encoder les labels en valeurs numériques
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label'])

data.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,label,label_encoded
0,-726.217224,68.541420,3.293398,12.205300,5.510278,13.667408,-2.983829,3.098029,-3.310813,-1.564384,...,-2.926855,0.013957,-0.490734,-0.570905,0.040399,-1.207217,-1.594982,-1.436487,neutral,5
1,-719.128296,70.201569,1.168397,13.122543,7.836950,14.411290,-4.111360,4.468973,-3.539367,-3.658607,...,-2.987673,0.409735,-0.484184,-1.398391,0.255203,-0.984978,-2.093061,-1.040791,neutral,5
2,-714.995728,69.689346,3.924564,11.924190,6.421723,11.011614,-2.878103,4.509558,-4.476109,-2.671550,...,-3.045955,-0.373294,-0.849145,-0.922105,-0.170320,-1.144423,-1.725613,-1.450561,neutral,5
3,-710.975281,67.564880,5.782241,13.230727,6.190846,12.628252,-1.675170,5.657494,-4.950634,-3.477545,...,-2.513405,-0.190276,-0.645949,-0.553919,0.459299,-1.580085,-1.647682,-1.509511,neutral,5
4,-759.921753,75.783524,6.023605,14.557394,6.454188,14.631508,-3.004551,4.620970,-5.200016,-0.707430,...,-2.835501,0.463746,-1.019167,-1.411441,0.350433,-1.519892,-1.250112,-0.613852,calm,1


In [43]:
data.shape

(1440, 42)

In [45]:
data.to_csv("../data/preprocessed/simple_processed_data.csv", index=False)

In [39]:
# Diviser les données
X = data.iloc[:, :-2].values  # Toutes les colonnes sauf 'label' et 'label_encoded'
y = data['label_encoded'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

## Advanced Encoding

In [53]:
def extract_features_advanced(file_path):
    # Charger le fichier audio
    audio, sr = librosa.load(file_path, sr=None)  
    
    # MFCCs : 100 coefficients
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=100)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    
    # Chroma STFT : Capturer l'énergie distribuée à travers les 12 tons
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)
    chroma_stft_scaled = np.mean(chroma_stft.T, axis=0)
    
    # Spectrogramme de puissance (Mel Spectrogram) : 128 valeurs
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram)
    mel_spectrogram_scaled = np.mean(mel_spectrogram_db.T, axis=0)
    
    # STFT (Short-Time Fourier Transform) : 1025 valeurs
    stft = np.abs(librosa.stft(audio))
    stft_scaled = np.mean(stft.T, axis=0)
    
    # Concaténer toutes les caractéristiques
    combined_features = np.hstack([mfccs_scaled, chroma_stft_scaled, mel_spectrogram_scaled, stft_scaled])
    
    return combined_features

#### Problème : caractéristiques inéquitables en terme de features --> STFT va prendre toute la place.

#### Solutions :
    - StandardScaler pour normaliser toutes les valeurs
    - Réduction de dimensionnalité pour les caractéristiques dominants

In [54]:
features = []
for file_path in file_paths:
    features.append(extract_features_advanced(file_path))

In [55]:
print("Taille du vecteur représentant un fichier :", len(features[1]))
print("Nombre de fichiers :", len(features))

Taille du vecteur représentant un fichier : 1265
Nombre de fichiers : 1440


In [56]:
# Convertir les données en DataFrame
data = pd.DataFrame(features)
data['label'] = labels

data.head()

,0,1,2,3,4,5,6,7,8,9,...,1256,1257,1258,1259,1260,1261,1262,1263,1264,label
0,-726.217224,68.541420,3.293398,12.205300,5.510278,13.667408,-2.983829,3.098029,-3.310813,-1.564384,...,0.000278,0.000283,0.000310,0.000303,0.000316,0.000304,0.000274,0.000288,0.000286,neutral
1,-719.128296,70.201569,1.168397,13.122543,7.836950,14.411290,-4.111360,4.468973,-3.539367,-3.658607,...,0.000290,0.000306,0.000310,0.000306,0.000302,0.000300,0.000285,0.000275,0.000242,neutral
2,-714.995728,69.689346,3.924564,11.924190,6.421723,11.011614,-2.878103,4.509558,-4.476109,-2.671550,...,0.000286,0.000255,0.000259,0.000288,0.000284,0.000284,0.000261,0.000258,0.000254,neutral
3,-710.975281,67.564880,5.782241,13.230727,6.190846,12.628252,-1.675170,5.657494,-4.950634,-3.477545,...,0.000257,0.000283,0.000289,0.000293,0.000280,0.000279,0.000285,0.000272,0.000221,neutral
4,-759.921753,75.783524,6.023605,14.557394,6.454188,14.631508,-3.004551,4.620970,-5.200016,-0.707430,...,0.000268,0.000268,0.000264,0.000277,0.000279,0.000279,0.000293,0.000264,0.000221,calm


In [57]:
# Encoder les labels en valeurs numériques
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label'])

data.head()

,0,1,2,3,4,5,6,7,8,9,...,1257,1258,1259,1260,1261,1262,1263,1264,label,label_encoded
0,-726.217224,68.541420,3.293398,12.205300,5.510278,13.667408,-2.983829,3.098029,-3.310813,-1.564384,...,0.000283,0.000310,0.000303,0.000316,0.000304,0.000274,0.000288,0.000286,neutral,5
1,-719.128296,70.201569,1.168397,13.122543,7.836950,14.411290,-4.111360,4.468973,-3.539367,-3.658607,...,0.000306,0.000310,0.000306,0.000302,0.000300,0.000285,0.000275,0.000242,neutral,5
2,-714.995728,69.689346,3.924564,11.924190,6.421723,11.011614,-2.878103,4.509558,-4.476109,-2.671550,...,0.000255,0.000259,0.000288,0.000284,0.000284,0.000261,0.000258,0.000254,neutral,5
3,-710.975281,67.564880,5.782241,13.230727,6.190846,12.628252,-1.675170,5.657494,-4.950634,-3.477545,...,0.000283,0.000289,0.000293,0.000280,0.000279,0.000285,0.000272,0.000221,neutral,5
4,-759.921753,75.783524,6.023605,14.557394,6.454188,14.631508,-3.004551,4.620970,-5.200016,-0.707430,...,0.000268,0.000264,0.000277,0.000279,0.000279,0.000293,0.000264,0.000221,calm,1


In [58]:
data.shape

(1440, 1267)

In [59]:
data.to_csv("../data/preprocessed/advanced_processed_data.csv", index=False)

In [60]:
# Diviser les données
X = data.iloc[:, :-2].values  # Toutes les colonnes sauf 'label' et 'label_encoded'
y = data['label_encoded'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [61]:
X_train.shape

(1152, 1265)